In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
import glob
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# Set TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Iterate through physical GPUs
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth cannot be modified after GPU has been initialized
        print(e)

# Set seaborn style
sns.set(rc={'axes.labelsize': 12, 'ytick.labelsize': 12, 'xtick.labelsize': 12, 'axes.titlesize': 15})


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
train_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Train")
test_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Test")
#test_simple_dir = pathlib.Path("/kaggle/input/papayadata/valid")

#df = pd.read_csv("/kaggle/input/clssessss/_classes.csv")

In [ ]:
class_names = os.listdir(train_dir)
class_names

In [ ]:
# train and test size
image_count_train = len(list(train_dir.glob('*/*.png'))) + len(list(train_dir.glob('*/*.jpeg'))) + len(list(train_dir.glob('*/*.jpg')))
print(image_count_train)

image_count_test = len(list(test_dir.glob('*/*.png'))) + len(list(test_dir.glob('*/*.jpeg'))) + len(list(test_dir.glob('*/*.jpg')))
print(image_count_test)


In [ ]:
import tensorflow as tf
from pathlib import Path

# Set TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Iterate through physical GPUs
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth cannot be modified after GPU has been initialized
        print(e)


# Your code to define class_names and train_dir goes here

# Function to plot an image
def plotImage(image_path):
    img = plt.imread(image_path)
    plt.imshow(img)
    plt.axis('off')

# Loop through each class and plot an image
for i in range(3): 
    class_images = list(train_dir.glob(class_names[i]+'/*.jpg')) + \
                   list(train_dir.glob(class_names[i]+'/*.png')) + \
                   list(train_dir.glob(class_names[i]+'/*.jpeg'))
    if class_images:  # Check if images exist for this class
        plt.subplot(3, 3, i + 1)
        image_path = str(class_images[0])
        if image_path.lower().endswith('.jpeg'):
            img = Image.open(image_path)
            plt.imshow(img)
        else:
            plotImage(image_path)
        plt.title(class_names[i])
        plt.grid()
    else:
        print(f"No images found for class {class_names[i]}")
plt.show()


In [ ]:
plt.figure(figsize=(12, 10))
for i in range(3): 
    # Get image paths for the current class
    image_paths = list(train_dir.glob(class_names[i]+'/*.jpg')) + \
                  list(train_dir.glob(class_names[i]+'/*.bmp')) + \
                  list(train_dir.glob(class_names[i]+'/*.png'))

    # Check if there are any images for the current class
    if not image_paths:
        print(f"No images found for class: {class_names[i]}")
        continue  # Skip to the next class

    # Plot the first image for the current class
    plt.subplot(4, 4, i + 1)
    plotImage(str(image_paths[0]))  # Plot the first image path
    plt.title(class_names[i])
    plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Directories for training and testing data
train_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Train")
test_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Test")

# Helper function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(120, 120)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return img, img_array

# Load images from all subdirectories (classes)
def load_images_from_all_classes(dir_path, sample_size=5):
    all_image_paths = []
    for subdir in dir_path.iterdir():
        if subdir.is_dir():
            class_images = list(subdir.glob("*.jpg"))
            all_image_paths.extend(class_images[:sample_size])  # Sample a few images from each class
    return all_image_paths

# Get a few images for visualization
train_images = load_images_from_all_classes(train_dir, sample_size=5)

# Apply different preprocessing techniques
def apply_preprocessing(img_array):
    # Resize
    resized = tf.image.resize(img_array, [64, 64])

    # Flip horizontally
    flipped = tf.image.flip_left_right(img_array)

    # Adjust brightness
    brightened = tf.image.adjust_brightness(img_array, delta=0.2)

    # Add Gaussian noise
    noise = tf.random.normal(shape=img_array.shape, mean=0.0, stddev=0.05)
    noisy = tf.clip_by_value(img_array + noise, 0.0, 1.0)

    # Rotation (90 degrees)
    rotated = tf.image.rot90(img_array)

    return resized, flipped, brightened, noisy, rotated

# Plot results
def visualize_preprocessing(image_paths):
    plt.figure(figsize=(35, 30))
    for idx, image_path in enumerate(image_paths):
        # Load image and preprocess
        img, img_array = load_and_preprocess_image(image_path)

        # Apply preprocessing
        resized, flipped, brightened, noisy, rotated = apply_preprocessing(img_array)

        # Plot original and preprocessed images
        plt.subplot(len(image_paths), 6, idx * 6 + 1)
        plt.imshow(img_array)  # Use the normalized NumPy array
        plt.title("Original")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 2)
        plt.imshow(resized.numpy())
        plt.title("Resized")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 3)
        plt.imshow(flipped.numpy())
        plt.title("Flipped")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 4)
        plt.imshow(brightened.numpy())
        plt.title("Brightened")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 5)
        plt.imshow(noisy.numpy())
        plt.title("Noisy")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 6)
        plt.imshow(rotated.numpy())
        plt.title("Rotated")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# Visualize preprocessing on train images
visualize_preprocessing(train_images)


In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Directories for training and testing data
train_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Train")
test_dir = pathlib.Path("/kaggle/input/dancceforrmfff/Test")

# Helper function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(120, 120)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return img, img_array

# Select a few images for visualization
train_images = list(train_dir.glob("*/*.jpg"))[:5]

# Apply different preprocessing techniques
def apply_preprocessing(img_array):
    # Resize
    resized = tf.image.resize(img_array, [64, 64])

    # Flip horizontally
    flipped = tf.image.flip_left_right(img_array)

    # Adjust brightness
    brightened = tf.image.adjust_brightness(img_array, delta=0.2)

    # Add Gaussian noise
    noise = tf.random.normal(shape=img_array.shape, mean=0.0, stddev=0.05)
    noisy = tf.clip_by_value(img_array + noise, 0.0, 1.0)

    # Rotation (90 degrees)
    rotated = tf.image.rot90(img_array)

    return resized, flipped, brightened, noisy, rotated

# Plot results
def visualize_preprocessing(image_paths):
    plt.figure(figsize=(20, 15))
    for idx, image_path in enumerate(image_paths):
        # Load image and preprocess
        img, img_array = load_and_preprocess_image(image_path)

        # Apply preprocessing
        resized, flipped, brightened, noisy, rotated = apply_preprocessing(img_array)

        # Plot original and preprocessed images
        plt.subplot(len(image_paths), 6, idx * 6 + 1)
        plt.imshow(img_array)  # Use the normalized NumPy array
        plt.title("Original")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 2)
        plt.imshow(resized.numpy())
        plt.title("Resized")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 3)
        plt.imshow(flipped.numpy())
        plt.title("Flipped")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 4)
        plt.imshow(brightened.numpy())
        plt.title("Brightened")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 5)
        plt.imshow(noisy.numpy())
        plt.title("Noisy")
        plt.axis("off")

        plt.subplot(len(image_paths), 6, idx * 6 + 6)
        plt.imshow(rotated.numpy())
        plt.title("Rotated")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# Visualize preprocessing on train images
visualize_preprocessing(train_images)


In [ ]:
print('Training samples:')
num_samples = 0
for cell in os.listdir(train_dir):
    num_cells = len(os.listdir(os.path.join(train_dir, cell)))
    num_samples += num_cells
    print('Cell: {:15s}  num samples: {:d}'.format(cell, num_cells))
print('Total training samples: {:d}\n'.format(num_samples))

print('Test samples:')
num_samples = 0
for cell in os.listdir(test_dir):
    num_cells = len(os.listdir(os.path.join(test_dir, cell)))
    num_samples += num_cells
    print('Cell: {:15s}  num samples: {:d}'.format(cell, num_cells))
print('Total test samples: {:d}'.format(num_samples))

In [ ]:
# function to plot the training/validation accuracies/losses.

def plot_learning(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    #axes[0].grid()
    axes[0].legend(['loss','val_loss'])
    axes[1].plot(history.history['accuracy'])   
    axes[1].plot(history.history['val_accuracy'])
    #axes[1].grid()
    axes[1].legend(['accuracy','val_accuracy'])

In [ ]:
# resolution of images
plt.imread(str(list(train_dir.glob(class_names[i]+'/*.jpg'))[0])).shape

In [ ]:
# defining some parameters for the loader

batch_size = 32
img_height = 120
img_width = 120

# lower resolution images reduce the definition/clarity of certain features in images.
# It can make it harder for CNN to learn the features required for classification or detection.
# working with 120 x 120 resolution images for now.

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Define loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=21,
    validation_split= 0.15,
    subset= 'training',
    image_size=(img_height,img_width),
    batch_size = batch_size
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=21,
    validation_split= 0.12,
    subset= 'validation',
    image_size=(img_height,img_width),
    batch_size = batch_size
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height,img_width),
    batch_size = batch_size
)

In [ ]:
tf.test.is_gpu_available()
#tf.test.gpu_device_name()

In [ ]:
import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import tensorflow as tf


In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_disable_xla_devices'

import tensorflow as tf


# INCEPTION V3

In [ ]:
# defining some parameters for the loader

batch_size = 32
img_height = 120
img_width = 120

# lower resolution images reduce the definition/clarity of certain features in images.
# It can make it harder for CNN to learn the features required for classification or detection.
# working with 120 x 120 resolution images for now.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Define input shape and number of classes
input_shape = (120, 120, 3)  # Define your input shape
num_classes = 3

# Define a more complex InceptionV3 model with additional layers
def create_minimal_inception_v3(input_shape=input_shape):
    """
    Builds a more complex InceptionV3 model for multi-class classification with additional dense layers up to 2048 units.
    """
    # Input layer
    input_tensor = Input(shape=input_shape)

    # Load the InceptionV3 model (using no pre-trained weights)
    base_model = InceptionV3(weights=None, include_top=False, input_shape=input_shape)(input_tensor)

    # Freeze InceptionV3 layers
    base_model.trainable = False  # Freeze layers initially

    # Add a Global Average Pooling layer
    x = GlobalAveragePooling2D()(base_model)

    # Add dense layers with increasing units and dropout for regularization
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(2048, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Add more dense layers for further capacity
    x = Dense(4096, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Add another dense layer
    x = Dense(2048, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Output layer
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Build model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

# Create the model
model = create_minimal_inception_v3()

# Compile the model with a higher learning rate
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# You can now train the model using `model.fit()` with the appropriate training and validation data.
# Add early stopping and model checkpoint callbacks



# Correct ModelCheckpoint usage by adding a filepath argument
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Assuming you have train_data, train_labels, val_data, and val_labels
# model.fit(datagen.flow(train_data, train_labels, batch_size=32), epochs=50, 
#           validation_data=(val_data, val_labels), 
#           callbacks=[early_stopping, checkpoint])


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Specify loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)



In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
import tensorflow as tf

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.2,           # Reduce learning rate by half
    patience=3,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=0.00001,          # Minimum learning rate allowed
    verbose=1             # Print a message when the learning rate is reduced
)
callbacks = [lr_reducer]

In [ ]:
import tensorflow as tf

# Disable graph optimization
tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
import tensorflow as tf

# Set GPU memory growth to avoid memory fragmentation issues
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


In [ ]:
import warnings

# Suppress the warning
warnings.filterwarnings("ignore", message="All log messages before absl::InitializeLog()")

# Your code here


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Specify loss function
loss = SparseCategoricalCrossentropy(from_logits=False)

# Compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

epochs = 100
history_model_v3 = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    #steps_per_epoch=300,
                    epochs=epochs,
                   # callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt

def plot_learning(history_model_v3):
    """
    Plots training & validation accuracy and loss with different colors.
    """
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_model_v3.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history_model_v3.history['val_accuracy'], label='Validation Accuracy', color='green')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_model_v3.history['loss'], label='Train Loss', color='red')
    plt.plot(history_model_v3.history['val_loss'], label='Validation Loss', color='purple')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    plt.show()

# Call the function with your model history
plot_learning(history_model_v3)


In [ ]:
plot_learning(history_model_CNN)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Evaluating performance on test set

results = model.evaluate(test_ds)

print("Loss of the model  is - test ", results[0])
print("Accuracy of the model is - test", results[1]*100, "%\n")


results = model.evaluate(val_ds)

print("Loss of the model  is - val ", results[0])
print("Accuracy of the model is - val", results[1]*100, "%\n")

results = model.evaluate(train_ds)

print("Loss of the model  is - train ", results[0])
print("Accuracy of the model is - train", results[1]*100, "%")

In [ ]:
from sklearn.metrics import classification_report

predictions = np.array([])
labels =  np.array([])
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x, verbose=0),axis=1)
    predictions = np.concatenate([predictions, y_pred])
    labels = np.concatenate([labels, y.numpy()])

In [ ]:
print(classification_report(labels,predictions,
                           target_names = ['china fan dance', 'Sword Dance', 'dragon dance images']))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Replace with actual class names)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of rows and columns for the grid
rows, cols = 4, 4  

# Take a batch of images and labels from the dataset
for images, labels in test_ds.take(1):  # Take only one batch
    plt.figure(figsize=(12, 12))  # Set figure size

    for i in range(min(rows * cols, len(images))):  # Ensure it doesn't exceed dataset size
        plt.subplot(rows, cols, i + 1)  # Create a 4-row, 4-column subplot
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert image to uint8 format
        plt.axis("off")  # Hide axes

        # Get model predictions
        predictions = model.predict(images[i][None, ...])  # Get prediction probabilities
        predicted_class = np.argmax(predictions)  # Get class index
        confidence = np.max(predictions) * 100  # Get confidence in percentage

        # Set title with class name and confidence
        plt.title(f"{class_names[predicted_class]}\n{confidence:.2f}%", fontsize=10, color="red")

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the images


In [ ]:
for images, labels in test_ds.take(2):
    plt.imshow(images[0].numpy().astype("uint8"))
    print(images.shape)
    print(labels.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Initialize empty lists for true labels and predictions
true_labels = []
predicted_labels = []

# Iterate through the test dataset to obtain predictions
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x), axis=1)
    true_labels.extend(y.numpy())
    predicted_labels.extend(y_pred)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
#print(classification_report(true_labels, predicted_labels,
#                            target_names=["china fan dance", "Sword Dance", "dragon dance images"]))

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display confusion matrix
print("\nConfusion Matrix:")
print(cm)


In [ ]:
def plot_confusion_matrix (cm):
    plt.figure(figsize = (10,10))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt = '', 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

def plot_confusion_matrix_3d(cm, class_names):
    """
    Plots a 3D surface plot for the confusion matrix.
    """
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create the X and Y grid for the confusion matrix
    x, y = np.meshgrid(np.arange(cm.shape[0]), np.arange(cm.shape[1]))

    # Plotting the surface
    ax.plot_surface(x, y, cm, cmap='Blues', edgecolor='black', linewidth=1)

    # Annotate the surface with text (values from the confusion matrix)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x[i, j], y[i, j], cm[i, j], str(cm[i, j]), color='black', fontsize=12, ha='center')

    # Set labels for axes
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_zlabel('Count')

    # Set ticks for X and Y axes based on class names
    ax.set_xticks(np.arange(cm.shape[1]))
    ax.set_yticks(np.arange(cm.shape[0]))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)

    # Set title
    ax.set_title('Confusion Matrix in 3D')

    # Show the plot
    plt.show()
#["china fan dance", "Sword Dance", "dragon dance images"]
# Example usage:
# Assuming you have a confusion matrix `cm` and class names in `class_names`
class_names = ['china fan dance', 'Sword Dance', 'dragon dance images']  # Modify with your actual class names
plot_confusion_matrix_3d(cm, class_names)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Modify based on your dataset)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of epochs for plotting (replace with actual epoch count)
epochs = np.arange(1, 11)  # Example: 10 epochs

# Create a larger plot
fig, ax = plt.subplots(figsize=(16, 12))  # Enlarged plot size

# Set labels for axes and title
ax.set_xlabel('Epochs', fontsize=14, weight='bold')
ax.set_ylabel('Confidence (%)', fontsize=14, weight='bold')
ax.set_title('Model Accuracy with Confidence Bubbles', fontsize=16, weight='bold')

# Variables to store bubble data
x_data = []
y_data = []
sizes = []  # Bubble sizes (confidence or any other measure)
colors = []  # Bubble colors (can represent confidence)

# Generate random data for demonstration
# Replace this section with real model predictions for each epoch
for epoch in epochs:
    for class_idx in range(len(class_names)):
        # Simulating confidence for each class in each epoch
        confidence = np.random.uniform(0.60, 1.00) * 100  # Random confidence between 60 and 100%
        
        # Store the data for plotting
        x_data.append(epoch)
        y_data.append(confidence)
        sizes.append(confidence * 25)  # Increased bubble size (scaled more)
        colors.append(confidence)  # Set color based on confidence

# Scatter plot with bubbles
scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='plasma', alpha=0.8, edgecolors='black', linewidth=1.5)

# Add color bar for confidence
cbar = plt.colorbar(scatter, ax=ax, label='Confidence %')
cbar.set_ticks([0, 25, 50, 75, 100])
cbar.ax.tick_params(labelsize=12)

# Adjust spacing for labels to avoid overlap and keep labels inside the plot
for i in range(len(x_data)):
    # Offset y-position to avoid going outside the plotting area
    label_x = x_data[i]
    label_y = y_data[i] + 5  # Adjust this offset if necessary

    # Check if the label is too close to the top or bottom and adjust
    if label_y > 100:  # If label goes above the upper limit
        label_y = 95  # Place label a little below 100
    elif label_y < 10:  # If label is too close to the bottom
        label_y = 15  # Place label a little above 0
    
    ax.text(label_x, label_y, f'{class_names[x_data[i] % len(class_names)]}\n{y_data[i]:.1f}%', 
            ha='center', fontsize=8, color='black', weight='bold', fontstyle='italic', 
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))  # Background for better readability

# Adjusting grid for better presentation
ax.set_xticks(epochs)
ax.set_yticks(np.arange(0, 110, 10))
ax.set_yticklabels(np.arange(0, 110, 10), fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Increase space between labels and bubbles
ax.margins(x=0.05, y=0.1)

# Tight layout to prevent overlap
plt.tight_layout(pad=5.0)  # Increased padding for better spacing

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles(history_model_v3):
    """
    Plots training and validation accuracy with bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v3.history['accuracy'],
        'val': history_model_v3.history['val_accuracy']
    }
    
    # Create a 2D plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for start, end in epoch_ranges:
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with Bubbles')
    ax.legend()
    
    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles(history_model_v3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles_separated(history_model_v3):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v3.history['accuracy'],
        'val': history_model_v3.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Variables to store bubble data
        x_data = []
        y_data = []
        sizes = []  # Bubble sizes (based on accuracy)
        colors = []  # Bubble colors (for visualization)
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

        # Scatter plot with bubbles
        scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add color bar for accuracy
        plt.colorbar(scatter, ax=ax, label='Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles_separated(history_model_v3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_solid_lines(history_model_v3):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100),
    using solid lines to represent the accuracies.
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v3.history['accuracy'],
        'val': history_model_v3.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Plot the lines for training and validation accuracy with new colors
        ax.plot(epoch_range, train_accuracy_range, label=f'Training Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='darkorange', linewidth=3)
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='mediumseagreen', linewidth=3)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_solid_lines(history_model_v3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module
from matplotlib import cm
from matplotlib.colors import Normalize

def plot_accuracy_with_3d_lines(history_model_v3):
    """
    Plots training and validation accuracy in separate 3D subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    Adds enhancements such as color gradients for a stunning 3D effect.
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v3.history['accuracy'],
        'val': history_model_v3.history['val_accuracy']
    }
    
    # Create a figure for the 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')  # 3D subplot
    
    # Set the colormap and normalize the values for color gradients
    cmap = cm.viridis  # You can change this to any other colormap such as 'plasma', 'inferno', etc.
    norm = Normalize(vmin=1, vmax=100)

    # Loop through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Color gradient based on epoch number
        train_colors = cmap(norm(epoch_range))  # Apply colormap to training accuracy
        val_colors = cmap(norm(epoch_range))    # Apply colormap to validation accuracy

        # 3D plot for training accuracy
        ax.plot(epoch_range, train_accuracy_range, zs=1, label=f'Training Accuracy (Epochs {start}-{end})', marker='o', color=train_colors[-1], linewidth=3, markersize=8)
        
        # 3D plot for validation accuracy
        ax.plot(epoch_range, val_accuracy_range, zs=2, label=f'Validation Accuracy (Epochs {start}-{end})', marker='^', color=val_colors[-1], linewidth=3, markersize=8)

    # Set labels with improved styling
    ax.set_xlabel('Epochs', fontsize=14, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold')
    ax.set_zlabel('Accuracy Level', fontsize=14, fontweight='bold')

    # Set title with improved styling
    ax.set_title('Training and Validation Accuracy in 3D', fontsize=16, fontweight='bold', color='darkblue')

    # Customize the grid and background for better aesthetics
    ax.grid(True, color='gray', linestyle='--', linewidth=0.5)
    ax.set_facecolor('whitesmoke')
    
    # Adjust the view angle for better clarity
    ax.view_init(30, 45)

    # Add a legend with better positioning
    ax.legend(loc='upper left', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_lines(history_model_v3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_accuracy_with_3d_bubbles(history_model_v3):
    """
    Plots training and validation accuracy with 3D bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v3.history['accuracy'],
        'val': history_model_v3.history['val_accuracy']
    }
    
    # Create a 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_zlabel('Accuracy Level')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    z_data = []  # Z-axis for different epoch ranges
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            z_data.append(idx + 1)  # Different z values based on epoch range (1, 2, 3, 4)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles (for train and validation accuracy)
        ax.plot(epoch_range, train_accuracy_range, zs=idx + 1, label=f'Train Accuracy (Epochs {start}-{end})', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, zs=idx + 1, label=f'Validation Accuracy (Epochs {start}-{end})', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, z_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with 3D Bubbles', fontsize=16, fontweight='bold')
    ax.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_bubbles(history_model_v3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_learning_3d(history_model_v3):
    """
    Plots training & validation accuracy and loss in 3D.
    """
    epochs = np.arange(1, len(history_model_v3.history['accuracy']) + 1)
    
    # Create 3D plot
    fig = plt.figure(figsize=(14, 10))

    # First 3D subplot for accuracy
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot(epochs, history_model_v3.history['accuracy'], zs=1, label='Train Accuracy', color='blue', linewidth=2)
    ax1.plot(epochs, history_model_v3.history['val_accuracy'], zs=2, label='Validation Accuracy', color='green', linewidth=2)
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.set_zlabel("Type")
    ax1.set_title("Training and Validation Accuracy in 3D")
    ax1.legend()

    # Second 3D subplot for loss
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot(epochs, history_model_v3.history['loss'], zs=1, label='Train Loss', color='red', linewidth=2)
    ax2.plot(epochs, history_model_v3.history['val_loss'], zs=2, label='Validation Loss', color='purple', linewidth=2)
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.set_zlabel("Type")
    ax2.set_title("Training and Validation Loss in 3D")
    ax2.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_learning_3d(history_model_v3)


# INCEPTION v4

In [ ]:
# defining some parameters for the loader

batch_size = 32
img_height = 120
img_width = 120

# lower resolution images reduce the definition/clarity of certain features in images.
# It can make it harder for CNN to learn the features required for classification or detection.
# working with 120 x 120 resolution images for now.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, GlobalAveragePooling2D, Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.applications import InceptionV4
import tensorflow as tf

# Define a more complex InceptionV4-based model
def create_improved_inception_model(input_shape=(120, 120, 3), num_classes=3):
    input_tensor = Input(shape=input_shape)

    # Resize to InceptionV4 input size (299x299)
    x = Lambda(lambda image: tf.image.resize(image, (299, 299)), output_shape=(299, 299, 3))(input_tensor)

    # Load InceptionV3 without top layers
    base_model = InceptionV4(weights=None, include_top=False, input_tensor=x)

    # Unfreeze more layers of the base model (not just the last few)
    for layer in base_model.layers[:]:
        layer.trainable = True  # Unfreeze all layers

    # Add additional layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Convert features into a single vector
    x = Dense(512, activation='relu')(x)  # Add a Dense layer with 512 units
    x = BatchNormalization()(x)  # Normalize the activations
    x = Dropout(0.5)(x)  # Add Dropout to reduce overfitting
    x = Dense(256, activation='relu')(x)  # Add another Dense layer with 256 units
    x = BatchNormalization()(x)  # Normalize again
    x = Dropout(0.5)(x)  # Another Dropout layer

    output_tensor = Dense(num_classes, activation='softmax')(x)  # Final classification layer

    # Create model
    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

# Define the number of classes
num_classes = 3

# Create and compile the model
model = create_improved_inception_model(input_shape=(120, 120, 3), num_classes=num_classes)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.summary()  # Optional: Check the model structure


In [ ]:
import tensorflow as tf

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.2,           # Reduce learning rate by half
    patience=5,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-5,          # Minimum learning rate allowed
    verbose=1             # Print a message when the learning rate is reduced
)
callbacks = [lr_reducer]

In [ ]:
import tensorflow as tf

# Disable graph optimization
tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import tensorflow as tf

# Set GPU memory growth to avoid memory fragmentation issues
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Specify loss function
loss = SparseCategoricalCrossentropy(from_logits=False)

# Compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

epochs = 100
history_model_v4 = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    #steps_per_epoch=100,
                    epochs=epochs,
                   # callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt

def plot_learning(history_model_v4):
    """
    Plots training & validation accuracy and loss with different colors.
    """
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_model_v4.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history_model_v4.history['val_accuracy'], label='Validation Accuracy', color='green')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_model_v4.history['loss'], label='Train Loss', color='red')
    plt.plot(history_model_v4.history['val_loss'], label='Validation Loss', color='purple')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    plt.show()

# Call the function with your model history
plot_learning(history_model_v4)


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Evaluating performance on test set

results = model.evaluate(test_ds)

print("Loss of the model  is - test ", results[0])
print("Accuracy of the model is - test", results[1]*100, "%\n")


results = model.evaluate(val_ds)

print("Loss of the model  is - val ", results[0])
print("Accuracy of the model is - val", results[1]*100, "%\n")

results = model.evaluate(train_ds)

print("Loss of the model  is - train ", results[0])
print("Accuracy of the model is - train", results[1]*100, "%")

In [ ]:
from sklearn.metrics import classification_report

predictions = np.array([])
labels =  np.array([])
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x, verbose=0),axis=1)
    predictions = np.concatenate([predictions, y_pred])
    labels = np.concatenate([labels, y.numpy()])

In [ ]:
print(classification_report(labels,predictions,
                           target_names = ['china fan dance', 'Sword Dance', 'dragon dance images']))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Replace with actual class names)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of rows and columns for the grid
rows, cols = 4, 4  

# Take a batch of images and labels from the dataset
for images, labels in test_ds.take(1):  # Take only one batch
    plt.figure(figsize=(12, 12))  # Set figure size

    for i in range(min(rows * cols, len(images))):  # Ensure it doesn't exceed dataset size
        plt.subplot(rows, cols, i + 1)  # Create a 4-row, 4-column subplot
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert image to uint8 format
        plt.axis("off")  # Hide axes

        # Get model predictions
        predictions = model.predict(images[i][None, ...])  # Get prediction probabilities
        predicted_class = np.argmax(predictions)  # Get class index
        confidence = np.max(predictions) * 100  # Get confidence in percentage

        # Set title with class name and confidence
        plt.title(f"{class_names[predicted_class]}\n{confidence:.2f}%", fontsize=10, color="red")

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the images


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Initialize empty lists for true labels and predictions
true_labels = []
predicted_labels = []

# Iterate through the test dataset to obtain predictions
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x), axis=1)
    true_labels.extend(y.numpy())
    predicted_labels.extend(y_pred)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
#print(classification_report(true_labels, predicted_labels,
#                            target_names=["china fan dance", "Sword Dance", "dragon dance images"]))

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display confusion matrix
print("\nConfusion Matrix:")
print(cm)


In [ ]:
def plot_confusion_matrix (cm):
    plt.figure(figsize = (10,10))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt = '', 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

def plot_confusion_matrix_3d(cm, class_names):
    """
    Plots a 3D surface plot for the confusion matrix.
    """
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create the X and Y grid for the confusion matrix
    x, y = np.meshgrid(np.arange(cm.shape[0]), np.arange(cm.shape[1]))

    # Plotting the surface
    ax.plot_surface(x, y, cm, cmap='Blues', edgecolor='black', linewidth=1)

    # Annotate the surface with text (values from the confusion matrix)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x[i, j], y[i, j], cm[i, j], str(cm[i, j]), color='black', fontsize=12, ha='center')

    # Set labels for axes
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_zlabel('Count')

    # Set ticks for X and Y axes based on class names
    ax.set_xticks(np.arange(cm.shape[1]))
    ax.set_yticks(np.arange(cm.shape[0]))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)

    # Set title
    ax.set_title('Confusion Matrix in 3D')

    # Show the plot
    plt.show()
#["china fan dance", "Sword Dance", "dragon dance images"]
# Example usage:
# Assuming you have a confusion matrix `cm` and class names in `class_names`
class_names = ['china fan dance', 'Sword Dance', 'dragon dance images']  # Modify with your actual class names
plot_confusion_matrix_3d(cm, class_names)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Modify based on your dataset)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of epochs for plotting (replace with actual epoch count)
epochs = np.arange(1, 11)  # Example: 10 epochs

# Create a larger plot
fig, ax = plt.subplots(figsize=(16, 12))  # Enlarged plot size

# Set labels for axes and title
ax.set_xlabel('Epochs', fontsize=14, weight='bold')
ax.set_ylabel('Confidence (%)', fontsize=14, weight='bold')
ax.set_title('Model Accuracy with Confidence Bubbles', fontsize=16, weight='bold')

# Variables to store bubble data
x_data = []
y_data = []
sizes = []  # Bubble sizes (confidence or any other measure)
colors = []  # Bubble colors (can represent confidence)

# Generate random data for demonstration
# Replace this section with real model predictions for each epoch
for epoch in epochs:
    for class_idx in range(len(class_names)):
        # Simulating confidence for each class in each epoch
        confidence = np.random.uniform(0.60, 1.00) * 100  # Random confidence between 60 and 100%
        
        # Store the data for plotting
        x_data.append(epoch)
        y_data.append(confidence)
        sizes.append(confidence * 25)  # Increased bubble size (scaled more)
        colors.append(confidence)  # Set color based on confidence

# Scatter plot with bubbles
scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='plasma', alpha=0.8, edgecolors='black', linewidth=1.5)

# Add color bar for confidence
cbar = plt.colorbar(scatter, ax=ax, label='Confidence %')
cbar.set_ticks([0, 25, 50, 75, 100])
cbar.ax.tick_params(labelsize=12)

# Adjust spacing for labels to avoid overlap and keep labels inside the plot
for i in range(len(x_data)):
    # Offset y-position to avoid going outside the plotting area
    label_x = x_data[i]
    label_y = y_data[i] + 5  # Adjust this offset if necessary

    # Check if the label is too close to the top or bottom and adjust
    if label_y > 100:  # If label goes above the upper limit
        label_y = 95  # Place label a little below 100
    elif label_y < 10:  # If label is too close to the bottom
        label_y = 15  # Place label a little above 0
    
    ax.text(label_x, label_y, f'{class_names[x_data[i] % len(class_names)]}\n{y_data[i]:.1f}%', 
            ha='center', fontsize=8, color='black', weight='bold', fontstyle='italic', 
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))  # Background for better readability

# Adjusting grid for better presentation
ax.set_xticks(epochs)
ax.set_yticks(np.arange(0, 110, 10))
ax.set_yticklabels(np.arange(0, 110, 10), fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Increase space between labels and bubbles
ax.margins(x=0.05, y=0.1)

# Tight layout to prevent overlap
plt.tight_layout(pad=5.0)  # Increased padding for better spacing

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles(history_model_v4):
    """
    Plots training and validation accuracy with bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v4.history['accuracy'],
        'val': history_model_v4.history['val_accuracy']
    }
    
    # Create a 2D plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for start, end in epoch_ranges:
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with Bubbles')
    ax.legend()
    
    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles(history_model_v4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles_separated(history_model_v4):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v4.history['accuracy'],
        'val': history_model_v4.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Variables to store bubble data
        x_data = []
        y_data = []
        sizes = []  # Bubble sizes (based on accuracy)
        colors = []  # Bubble colors (for visualization)
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

        # Scatter plot with bubbles
        scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add color bar for accuracy
        plt.colorbar(scatter, ax=ax, label='Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles_separated(history_model_v4)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_solid_lines(history_model_v4):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100),
    using solid lines to represent the accuracies.
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v4.history['accuracy'],
        'val': history_model_v4.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Plot the lines for training and validation accuracy with new colors
        ax.plot(epoch_range, train_accuracy_range, label=f'Training Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='darkorange', linewidth=3)
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='mediumseagreen', linewidth=3)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_solid_lines(history_model_v4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module
from matplotlib import cm
from matplotlib.colors import Normalize

def plot_accuracy_with_3d_lines(history_model_v4):
    """
    Plots training and validation accuracy in separate 3D subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    Adds enhancements such as color gradients for a stunning 3D effect.
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v4.history['accuracy'],
        'val': history_model_v4.history['val_accuracy']
    }
    
    # Create a figure for the 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')  # 3D subplot
    
    # Set the colormap and normalize the values for color gradients
    cmap = cm.viridis  # You can change this to any other colormap such as 'plasma', 'inferno', etc.
    norm = Normalize(vmin=1, vmax=100)

    # Loop through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Color gradient based on epoch number
        train_colors = cmap(norm(epoch_range))  # Apply colormap to training accuracy
        val_colors = cmap(norm(epoch_range))    # Apply colormap to validation accuracy

        # 3D plot for training accuracy
        ax.plot(epoch_range, train_accuracy_range, zs=1, label=f'Training Accuracy (Epochs {start}-{end})', marker='o', color=train_colors[-1], linewidth=3, markersize=8)
        
        # 3D plot for validation accuracy
        ax.plot(epoch_range, val_accuracy_range, zs=2, label=f'Validation Accuracy (Epochs {start}-{end})', marker='^', color=val_colors[-1], linewidth=3, markersize=8)

    # Set labels with improved styling
    ax.set_xlabel('Epochs', fontsize=14, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold')
    ax.set_zlabel('Accuracy Level', fontsize=14, fontweight='bold')

    # Set title with improved styling
    ax.set_title('Training and Validation Accuracy in 3D', fontsize=16, fontweight='bold', color='darkblue')

    # Customize the grid and background for better aesthetics
    ax.grid(True, color='gray', linestyle='--', linewidth=0.5)
    ax.set_facecolor('whitesmoke')
    
    # Adjust the view angle for better clarity
    ax.view_init(30, 45)

    # Add a legend with better positioning
    ax.legend(loc='upper left', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_lines(history_model_v4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_accuracy_with_3d_bubbles(history_model_v4):
    """
    Plots training and validation accuracy with 3D bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_v4.history['accuracy'],
        'val': history_model_v4.history['val_accuracy']
    }
    
    # Create a 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_zlabel('Accuracy Level')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    z_data = []  # Z-axis for different epoch ranges
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            z_data.append(idx + 1)  # Different z values based on epoch range (1, 2, 3, 4)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles (for train and validation accuracy)
        ax.plot(epoch_range, train_accuracy_range, zs=idx + 1, label=f'Train Accuracy (Epochs {start}-{end})', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, zs=idx + 1, label=f'Validation Accuracy (Epochs {start}-{end})', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, z_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with 3D Bubbles', fontsize=16, fontweight='bold')
    ax.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_bubbles(history_model_v4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_learning_3d(history_model_v4):
    """
    Plots training & validation accuracy and loss in 3D.
    """
    epochs = np.arange(1, len(history_model_v4.history['accuracy']) + 1)
    
    # Create 3D plot
    fig = plt.figure(figsize=(14, 10))

    # First 3D subplot for accuracy
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot(epochs, history_model_v4.history['accuracy'], zs=1, label='Train Accuracy', color='blue', linewidth=2)
    ax1.plot(epochs, history_model_v4.history['val_accuracy'], zs=2, label='Validation Accuracy', color='green', linewidth=2)
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.set_zlabel("Type")
    ax1.set_title("Training and Validation Accuracy in 3D")
    ax1.legend()

    # Second 3D subplot for loss
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot(epochs, history_model_v4.history['loss'], zs=1, label='Train Loss', color='red', linewidth=2)
    ax2.plot(epochs, history_model_v4.history['val_loss'], zs=2, label='Validation Loss', color='purple', linewidth=2)
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.set_zlabel("Type")
    ax2.set_title("Training and Validation Loss in 3D")
    ax2.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_learning_3d(history_model_v4)


# RESNET50

In [ ]:
# defining some parameters for the loader

batch_size = 32
img_height = 120
img_width = 120

# lower resolution images reduce the definition/clarity of certain features in images.
# It can make it harder for CNN to learn the features required for classification or detection.
# working with 120 x 120 resolution images for now.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization

# Define the number of classes
num_classes = 3

# Define input dimensions (adjust according to your data)
img_height, img_width = 120, 120  # Example dimensions

# Load ResNet50 base model without top layers
resnet_base = ResNet50(weights=None, include_top=False)

# Define input tensor with specific shape
input_tensor = Input(shape=(img_height, img_width, 3))

# Add ResNet50 base model
x = resnet_base(input_tensor)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
x = GlobalAveragePooling2D()(x)

# Add Dense layers with BatchNormalization and Dropout for regularization
x = Dense(1024, activation='relu')(x)  # Increased units for more capacity
x = BatchNormalization()(x)  # BatchNormalization to stabilize training
x = Dropout(0.3)(x)  # Higher dropout to reduce overfitting

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.3)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.2)(x)

# Optional: Add more layers for additional capacity
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.2)(x)

# Output layer
output_tensor = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Print model summary
model.summary()


In [ ]:
import tensorflow as tf

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.2,           # Reduce learning rate by half
    patience=5,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-5,          # Minimum learning rate allowed
    verbose=1             # Print a message when the learning rate is reduced
)
callbacks = [lr_reducer]

In [ ]:
import tensorflow as tf

# Disable graph optimization
tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import tensorflow as tf

# Set GPU memory growth to avoid memory fragmentation issues
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Specify loss function
loss = SparseCategoricalCrossentropy(from_logits=False)

# Compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

epochs = 100
history_model_net = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    #steps_per_epoch=100,
                    epochs=epochs,
                   # callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt

def plot_learning(history_model_net):
    """
    Plots training & validation accuracy and loss with different colors.
    """
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_model_net.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history_model_net.history['val_accuracy'], label='Validation Accuracy', color='green')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_model_net.history['loss'], label='Train Loss', color='red')
    plt.plot(history_model_net.history['val_loss'], label='Validation Loss', color='purple')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    plt.show()

# Call the function with your model history
plot_learning(history_model_net)


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Evaluating performance on test set

results = model.evaluate(test_ds)

print("Loss of the model  is - test ", results[0])
print("Accuracy of the model is - test", results[1]*100, "%\n")


results = model.evaluate(val_ds)

print("Loss of the model  is - val ", results[0])
print("Accuracy of the model is - val", results[1]*100, "%\n")

results = model.evaluate(train_ds)

print("Loss of the model  is - train ", results[0])
print("Accuracy of the model is - train", results[1]*100, "%")

In [ ]:
from sklearn.metrics import classification_report

predictions = np.array([])
labels =  np.array([])
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x, verbose=0),axis=1)
    predictions = np.concatenate([predictions, y_pred])
    labels = np.concatenate([labels, y.numpy()])

In [ ]:
print(classification_report(labels,predictions,
                           target_names = ['china fan dance', 'Sword Dance', 'dragon dance images']))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Replace with actual class names)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of rows and columns for the grid
rows, cols = 4, 4  

# Take a batch of images and labels from the dataset
for images, labels in test_ds.take(1):  # Take only one batch
    plt.figure(figsize=(12, 12))  # Set figure size

    for i in range(min(rows * cols, len(images))):  # Ensure it doesn't exceed dataset size
        plt.subplot(rows, cols, i + 1)  # Create a 4-row, 4-column subplot
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert image to uint8 format
        plt.axis("off")  # Hide axes

        # Get model predictions
        predictions = model.predict(images[i][None, ...])  # Get prediction probabilities
        predicted_class = np.argmax(predictions)  # Get class index
        confidence = np.max(predictions) * 100  # Get confidence in percentage

        # Set title with class name and confidence
        plt.title(f"{class_names[predicted_class]}\n{confidence:.2f}%", fontsize=10, color="red")

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the images


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Initialize empty lists for true labels and predictions
true_labels = []
predicted_labels = []

# Iterate through the test dataset to obtain predictions
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x), axis=1)
    true_labels.extend(y.numpy())
    predicted_labels.extend(y_pred)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
#print(classification_report(true_labels, predicted_labels,
#                            target_names=["china fan dance", "Sword Dance", "dragon dance images"]))

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display confusion matrix
print("\nConfusion Matrix:")
print(cm)


In [ ]:
def plot_confusion_matrix (cm):
    plt.figure(figsize = (10,10))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt = '', 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

def plot_confusion_matrix_3d(cm, class_names):
    """
    Plots a 3D surface plot for the confusion matrix.
    """
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create the X and Y grid for the confusion matrix
    x, y = np.meshgrid(np.arange(cm.shape[0]), np.arange(cm.shape[1]))

    # Plotting the surface
    ax.plot_surface(x, y, cm, cmap='Blues', edgecolor='black', linewidth=1)

    # Annotate the surface with text (values from the confusion matrix)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x[i, j], y[i, j], cm[i, j], str(cm[i, j]), color='black', fontsize=12, ha='center')

    # Set labels for axes
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_zlabel('Count')

    # Set ticks for X and Y axes based on class names
    ax.set_xticks(np.arange(cm.shape[1]))
    ax.set_yticks(np.arange(cm.shape[0]))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)

    # Set title
    ax.set_title('Confusion Matrix in 3D')

    # Show the plot
    plt.show()
#["china fan dance", "Sword Dance", "dragon dance images"]
# Example usage:
# Assuming you have a confusion matrix `cm` and class names in `class_names`
class_names = ['china fan dance', 'Sword Dance', 'dragon dance images']  # Modify with your actual class names
plot_confusion_matrix_3d(cm, class_names)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Modify based on your dataset)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of epochs for plotting (replace with actual epoch count)
epochs = np.arange(1, 11)  # Example: 10 epochs

# Create a larger plot
fig, ax = plt.subplots(figsize=(16, 12))  # Enlarged plot size

# Set labels for axes and title
ax.set_xlabel('Epochs', fontsize=14, weight='bold')
ax.set_ylabel('Confidence (%)', fontsize=14, weight='bold')
ax.set_title('Model Accuracy with Confidence Bubbles', fontsize=16, weight='bold')

# Variables to store bubble data
x_data = []
y_data = []
sizes = []  # Bubble sizes (confidence or any other measure)
colors = []  # Bubble colors (can represent confidence)

# Generate random data for demonstration
# Replace this section with real model predictions for each epoch
for epoch in epochs:
    for class_idx in range(len(class_names)):
        # Simulating confidence for each class in each epoch
        confidence = np.random.uniform(0.60, 1.00) * 100  # Random confidence between 60 and 100%
        
        # Store the data for plotting
        x_data.append(epoch)
        y_data.append(confidence)
        sizes.append(confidence * 25)  # Increased bubble size (scaled more)
        colors.append(confidence)  # Set color based on confidence

# Scatter plot with bubbles
scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='plasma', alpha=0.8, edgecolors='black', linewidth=1.5)

# Add color bar for confidence
cbar = plt.colorbar(scatter, ax=ax, label='Confidence %')
cbar.set_ticks([0, 25, 50, 75, 100])
cbar.ax.tick_params(labelsize=12)

# Adjust spacing for labels to avoid overlap and keep labels inside the plot
for i in range(len(x_data)):
    # Offset y-position to avoid going outside the plotting area
    label_x = x_data[i]
    label_y = y_data[i] + 5  # Adjust this offset if necessary

    # Check if the label is too close to the top or bottom and adjust
    if label_y > 100:  # If label goes above the upper limit
        label_y = 95  # Place label a little below 100
    elif label_y < 10:  # If label is too close to the bottom
        label_y = 15  # Place label a little above 0
    
    ax.text(label_x, label_y, f'{class_names[x_data[i] % len(class_names)]}\n{y_data[i]:.1f}%', 
            ha='center', fontsize=8, color='black', weight='bold', fontstyle='italic', 
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))  # Background for better readability

# Adjusting grid for better presentation
ax.set_xticks(epochs)
ax.set_yticks(np.arange(0, 110, 10))
ax.set_yticklabels(np.arange(0, 110, 10), fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Increase space between labels and bubbles
ax.margins(x=0.05, y=0.1)

# Tight layout to prevent overlap
plt.tight_layout(pad=5.0)  # Increased padding for better spacing

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles(history_model_net):
    """
    Plots training and validation accuracy with bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a 2D plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for start, end in epoch_ranges:
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with Bubbles')
    ax.legend()
    
    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles_separated(history_model_net):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Variables to store bubble data
        x_data = []
        y_data = []
        sizes = []  # Bubble sizes (based on accuracy)
        colors = []  # Bubble colors (for visualization)
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

        # Scatter plot with bubbles
        scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add color bar for accuracy
        plt.colorbar(scatter, ax=ax, label='Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles_separated(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_solid_lines(history_model_net):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100),
    using solid lines to represent the accuracies.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Plot the lines for training and validation accuracy with new colors
        ax.plot(epoch_range, train_accuracy_range, label=f'Training Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='darkorange', linewidth=3)
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='mediumseagreen', linewidth=3)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_solid_lines(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module
from matplotlib import cm
from matplotlib.colors import Normalize

def plot_accuracy_with_3d_lines(history_model_net):
    """
    Plots training and validation accuracy in separate 3D subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    Adds enhancements such as color gradients for a stunning 3D effect.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a figure for the 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')  # 3D subplot
    
    # Set the colormap and normalize the values for color gradients
    cmap = cm.viridis  # You can change this to any other colormap such as 'plasma', 'inferno', etc.
    norm = Normalize(vmin=1, vmax=100)

    # Loop through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Color gradient based on epoch number
        train_colors = cmap(norm(epoch_range))  # Apply colormap to training accuracy
        val_colors = cmap(norm(epoch_range))    # Apply colormap to validation accuracy

        # 3D plot for training accuracy
        ax.plot(epoch_range, train_accuracy_range, zs=1, label=f'Training Accuracy (Epochs {start}-{end})', marker='o', color=train_colors[-1], linewidth=3, markersize=8)
        
        # 3D plot for validation accuracy
        ax.plot(epoch_range, val_accuracy_range, zs=2, label=f'Validation Accuracy (Epochs {start}-{end})', marker='^', color=val_colors[-1], linewidth=3, markersize=8)

    # Set labels with improved styling
    ax.set_xlabel('Epochs', fontsize=14, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold')
    ax.set_zlabel('Accuracy Level', fontsize=14, fontweight='bold')

    # Set title with improved styling
    ax.set_title('Training and Validation Accuracy in 3D', fontsize=16, fontweight='bold', color='darkblue')

    # Customize the grid and background for better aesthetics
    ax.grid(True, color='gray', linestyle='--', linewidth=0.5)
    ax.set_facecolor('whitesmoke')
    
    # Adjust the view angle for better clarity
    ax.view_init(30, 45)

    # Add a legend with better positioning
    ax.legend(loc='upper left', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_lines(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_accuracy_with_3d_bubbles(history_model_net):
    """
    Plots training and validation accuracy with 3D bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_zlabel('Accuracy Level')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    z_data = []  # Z-axis for different epoch ranges
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            z_data.append(idx + 1)  # Different z values based on epoch range (1, 2, 3, 4)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles (for train and validation accuracy)
        ax.plot(epoch_range, train_accuracy_range, zs=idx + 1, label=f'Train Accuracy (Epochs {start}-{end})', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, zs=idx + 1, label=f'Validation Accuracy (Epochs {start}-{end})', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, z_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with 3D Bubbles', fontsize=16, fontweight='bold')
    ax.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_bubbles(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_learning_3d(history_model_net):
    """
    Plots training & validation accuracy and loss in 3D.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Create 3D plot
    fig = plt.figure(figsize=(14, 10))

    # First 3D subplot for accuracy
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot(epochs, history_model_net.history['accuracy'], zs=1, label='Train Accuracy', color='blue', linewidth=2)
    ax1.plot(epochs, history_model_net.history['val_accuracy'], zs=2, label='Validation Accuracy', color='green', linewidth=2)
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.set_zlabel("Type")
    ax1.set_title("Training and Validation Accuracy in 3D")
    ax1.legend()

    # Second 3D subplot for loss
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot(epochs, history_model_net.history['loss'], zs=1, label='Train Loss', color='red', linewidth=2)
    ax2.plot(epochs, history_model_net.history['val_loss'], zs=2, label='Validation Loss', color='purple', linewidth=2)
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.set_zlabel("Type")
    ax2.set_title("Training and Validation Loss in 3D")
    ax2.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_learning_3d(history_model_net)


# MOBILE NET

In [ ]:
# defining some parameters for the loader

batch_size = 32
img_height = 120
img_width = 120

# lower resolution images reduce the definition/clarity of certain features in images.
# It can make it harder for CNN to learn the features required for classification or detection.
# working with 120 x 120 resolution images for now.

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization

# Define the number of classes
num_classes = 3

# Define input dimensions (adjust according to your data)
img_height, img_width = 120, 120  # Example dimensions

# Load MobileNet base model without top layers
mobilenet_base = MobileNet(weights=None, include_top=False)

# Define input tensor with specific shape
input_tensor = Input(shape=(img_height, img_width, 3))

# Add MobileNet base model
x = mobilenet_base(input_tensor)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
x = GlobalAveragePooling2D()(x)

# Add Dense layers with BatchNormalization and Dropout for regularization
x = Dense(1024, activation='relu')(x)  # Increased units for more capacity
x = BatchNormalization()(x)  # BatchNormalization to stabilize training
x = Dropout(0.4)(x)  # Increased dropout to prevent overfitting

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.4)(x)  # Increased dropout

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.3)(x)  # Moderate dropout

# Optional: Add more layers for additional capacity
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)  # BatchNormalization
x = Dropout(0.3)(x)  # Moderate dropout

# Output layer
output_tensor = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Print model summary
model.summary()


In [ ]:
import tensorflow as tf

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.2,           # Reduce learning rate by half
    patience=5,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-5,          # Minimum learning rate allowed
    verbose=1             # Print a message when the learning rate is reduced
)
callbacks = [lr_reducer]

In [ ]:
import tensorflow as tf

# Disable graph optimization
tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import tensorflow as tf

# Set GPU memory growth to avoid memory fragmentation issues
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    # Set memory growth for GPU
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize Adam optimizer
optimizer = Adam()

# Specify loss function
loss = SparseCategoricalCrossentropy(from_logits=False)

# Compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

epochs = 100
history_model_net = model.fit(
                    train_ds,
                    validation_data=val_ds,
                    #steps_per_epoch=100,
                    epochs=epochs,
                   # callbacks=callbacks
)


In [ ]:
import matplotlib.pyplot as plt

def plot_learning(history_model_net):
    """
    Plots training & validation accuracy and loss with different colors.
    """
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_model_net.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(history_model_net.history['val_accuracy'], label='Validation Accuracy', color='green')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_model_net.history['loss'], label='Train Loss', color='red')
    plt.plot(history_model_net.history['val_loss'], label='Validation Loss', color='purple')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    plt.show()

# Call the function with your model history
plot_learning(history_model_net)


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
# Evaluating performance on test set

results = model.evaluate(test_ds)

print("Loss of the model  is - test ", results[0])
print("Accuracy of the model is - test", results[1]*100, "%\n")


results = model.evaluate(val_ds)

print("Loss of the model  is - val ", results[0])
print("Accuracy of the model is - val", results[1]*100, "%\n")

results = model.evaluate(train_ds)

print("Loss of the model  is - train ", results[0])
print("Accuracy of the model is - train", results[1]*100, "%")

In [ ]:
from sklearn.metrics import classification_report

predictions = np.array([])
labels =  np.array([])
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x, verbose=0),axis=1)
    predictions = np.concatenate([predictions, y_pred])
    labels = np.concatenate([labels, y.numpy()])

In [ ]:
print(classification_report(labels,predictions,
                           target_names = ['china fan dance', 'Sword Dance', 'dragon dance images']))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Replace with actual class names)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of rows and columns for the grid
rows, cols = 4, 4  

# Take a batch of images and labels from the dataset
for images, labels in test_ds.take(1):  # Take only one batch
    plt.figure(figsize=(12, 12))  # Set figure size

    for i in range(min(rows * cols, len(images))):  # Ensure it doesn't exceed dataset size
        plt.subplot(rows, cols, i + 1)  # Create a 4-row, 4-column subplot
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert image to uint8 format
        plt.axis("off")  # Hide axes

        # Get model predictions
        predictions = model.predict(images[i][None, ...])  # Get prediction probabilities
        predicted_class = np.argmax(predictions)  # Get class index
        confidence = np.max(predictions) * 100  # Get confidence in percentage

        # Set title with class name and confidence
        plt.title(f"{class_names[predicted_class]}\n{confidence:.2f}%", fontsize=10, color="red")

    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()  # Display the images


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Initialize empty lists for true labels and predictions
true_labels = []
predicted_labels = []

# Iterate through the test dataset to obtain predictions
for x, y in val_ds:
    y_pred = np.argmax(model.predict(x), axis=1)
    true_labels.extend(y.numpy())
    predicted_labels.extend(y_pred)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Print classification report
#print(classification_report(true_labels, predicted_labels,
#                            target_names=["china fan dance", "Sword Dance", "dragon dance images"]))

# Create confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Display confusion matrix
print("\nConfusion Matrix:")
print(cm)


In [ ]:
def plot_confusion_matrix (cm):
    plt.figure(figsize = (10,10))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt = '', 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

def plot_confusion_matrix_3d(cm, class_names):
    """
    Plots a 3D surface plot for the confusion matrix.
    """
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Create the X and Y grid for the confusion matrix
    x, y = np.meshgrid(np.arange(cm.shape[0]), np.arange(cm.shape[1]))

    # Plotting the surface
    ax.plot_surface(x, y, cm, cmap='Blues', edgecolor='black', linewidth=1)

    # Annotate the surface with text (values from the confusion matrix)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x[i, j], y[i, j], cm[i, j], str(cm[i, j]), color='black', fontsize=12, ha='center')

    # Set labels for axes
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_zlabel('Count')

    # Set ticks for X and Y axes based on class names
    ax.set_xticks(np.arange(cm.shape[1]))
    ax.set_yticks(np.arange(cm.shape[0]))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)

    # Set title
    ax.set_title('Confusion Matrix in 3D')

    # Show the plot
    plt.show()
#["china fan dance", "Sword Dance", "dragon dance images"]
# Example usage:
# Assuming you have a confusion matrix `cm` and class names in `class_names`
class_names = ['china fan dance', 'Sword Dance', 'dragon dance images']  # Modify with your actual class names
plot_confusion_matrix_3d(cm, class_names)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define class names (Modify based on your dataset)
class_names = ["china fan dance", "Sword Dance", "dragon dance images"]  # Modify based on your dataset

# Number of epochs for plotting (replace with actual epoch count)
epochs = np.arange(1, 11)  # Example: 10 epochs

# Create a larger plot
fig, ax = plt.subplots(figsize=(16, 12))  # Enlarged plot size

# Set labels for axes and title
ax.set_xlabel('Epochs', fontsize=14, weight='bold')
ax.set_ylabel('Confidence (%)', fontsize=14, weight='bold')
ax.set_title('Model Accuracy with Confidence Bubbles', fontsize=16, weight='bold')

# Variables to store bubble data
x_data = []
y_data = []
sizes = []  # Bubble sizes (confidence or any other measure)
colors = []  # Bubble colors (can represent confidence)

# Generate random data for demonstration
# Replace this section with real model predictions for each epoch
for epoch in epochs:
    for class_idx in range(len(class_names)):
        # Simulating confidence for each class in each epoch
        confidence = np.random.uniform(0.60, 1.00) * 100  # Random confidence between 60 and 100%
        
        # Store the data for plotting
        x_data.append(epoch)
        y_data.append(confidence)
        sizes.append(confidence * 25)  # Increased bubble size (scaled more)
        colors.append(confidence)  # Set color based on confidence

# Scatter plot with bubbles
scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='plasma', alpha=0.8, edgecolors='black', linewidth=1.5)

# Add color bar for confidence
cbar = plt.colorbar(scatter, ax=ax, label='Confidence %')
cbar.set_ticks([0, 25, 50, 75, 100])
cbar.ax.tick_params(labelsize=12)

# Adjust spacing for labels to avoid overlap and keep labels inside the plot
for i in range(len(x_data)):
    # Offset y-position to avoid going outside the plotting area
    label_x = x_data[i]
    label_y = y_data[i] + 5  # Adjust this offset if necessary

    # Check if the label is too close to the top or bottom and adjust
    if label_y > 100:  # If label goes above the upper limit
        label_y = 95  # Place label a little below 100
    elif label_y < 10:  # If label is too close to the bottom
        label_y = 15  # Place label a little above 0
    
    ax.text(label_x, label_y, f'{class_names[x_data[i] % len(class_names)]}\n{y_data[i]:.1f}%', 
            ha='center', fontsize=8, color='black', weight='bold', fontstyle='italic', 
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7))  # Background for better readability

# Adjusting grid for better presentation
ax.set_xticks(epochs)
ax.set_yticks(np.arange(0, 110, 10))
ax.set_yticklabels(np.arange(0, 110, 10), fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Increase space between labels and bubbles
ax.margins(x=0.05, y=0.1)

# Tight layout to prevent overlap
plt.tight_layout(pad=5.0)  # Increased padding for better spacing

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles(history_model_net):
    """
    Plots training and validation accuracy with bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a 2D plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for start, end in epoch_ranges:
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with Bubbles')
    ax.legend()
    
    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_bubbles_separated(history_model_net):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Variables to store bubble data
        x_data = []
        y_data = []
        sizes = []  # Bubble sizes (based on accuracy)
        colors = []  # Bubble colors (for visualization)
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles
        ax.plot(epoch_range, train_accuracy_range, label=f'Train Accuracy Epochs {start}-{end}', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy Epochs {start}-{end}', marker='x', linestyle='-', color='green')

        # Scatter plot with bubbles
        scatter = ax.scatter(x_data, y_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add color bar for accuracy
        plt.colorbar(scatter, ax=ax, label='Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_bubbles_separated(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_with_solid_lines(history_model_net):
    """
    Plots training and validation accuracy in separate subplots for each epoch range (1-25, 26-50, 51-75, 76-100),
    using solid lines to represent the accuracies.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create subplots for each epoch range
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))  # 2x2 grid for 4 subplots
    axes = axes.flatten()  # Flatten to easily iterate over

    # Iterate through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        ax = axes[idx]
        
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Plot the lines for training and validation accuracy with new colors
        ax.plot(epoch_range, train_accuracy_range, label=f'Training Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='darkorange', linewidth=3)
        ax.plot(epoch_range, val_accuracy_range, label=f'Validation Accuracy (Epochs {start}-{end})', linestyle='-', marker='o', color='mediumseagreen', linewidth=3)

        # Set labels for axes
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Accuracy')
        
        # Add title and legend
        ax.set_title(f'Training and Validation Accuracy (Epochs {start}-{end})')
        ax.legend()

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_solid_lines(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module
from matplotlib import cm
from matplotlib.colors import Normalize

def plot_accuracy_with_3d_lines(history_model_net):
    """
    Plots training and validation accuracy in separate 3D subplots for each epoch range (1-25, 26-50, 51-75, 76-100).
    Adds enhancements such as color gradients for a stunning 3D effect.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a figure for the 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')  # 3D subplot
    
    # Set the colormap and normalize the values for color gradients
    cmap = cm.viridis  # You can change this to any other colormap such as 'plasma', 'inferno', etc.
    norm = Normalize(vmin=1, vmax=100)

    # Loop through each epoch range and plot on corresponding subplot
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Color gradient based on epoch number
        train_colors = cmap(norm(epoch_range))  # Apply colormap to training accuracy
        val_colors = cmap(norm(epoch_range))    # Apply colormap to validation accuracy

        # 3D plot for training accuracy
        ax.plot(epoch_range, train_accuracy_range, zs=1, label=f'Training Accuracy (Epochs {start}-{end})', marker='o', color=train_colors[-1], linewidth=3, markersize=8)
        
        # 3D plot for validation accuracy
        ax.plot(epoch_range, val_accuracy_range, zs=2, label=f'Validation Accuracy (Epochs {start}-{end})', marker='^', color=val_colors[-1], linewidth=3, markersize=8)

    # Set labels with improved styling
    ax.set_xlabel('Epochs', fontsize=14, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold')
    ax.set_zlabel('Accuracy Level', fontsize=14, fontweight='bold')

    # Set title with improved styling
    ax.set_title('Training and Validation Accuracy in 3D', fontsize=16, fontweight='bold', color='darkblue')

    # Customize the grid and background for better aesthetics
    ax.grid(True, color='gray', linestyle='--', linewidth=0.5)
    ax.set_facecolor('whitesmoke')
    
    # Adjust the view angle for better clarity
    ax.view_init(30, 45)

    # Add a legend with better positioning
    ax.legend(loc='upper left', fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_lines(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_accuracy_with_3d_bubbles(history_model_net):
    """
    Plots training and validation accuracy with 3D bubbles for different epoch ranges.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Split the epochs and accuracy into different ranges (1-25, 26-50, 51-75, 76-100)
    epoch_ranges = [(1, 25), (26, 50), (51, 75), (76, 100)]
    accuracies = {
        'train': history_model_net.history['accuracy'],
        'val': history_model_net.history['val_accuracy']
    }
    
    # Create a 3D plot
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set labels for axes
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.set_zlabel('Accuracy Level')
    
    # Variables to store bubble data
    x_data = []
    y_data = []
    z_data = []  # Z-axis for different epoch ranges
    sizes = []  # Bubble sizes (based on accuracy)
    colors = []  # Bubble colors (for visualization)
    
    # Plot training accuracy with bubbles and lines for each epoch range
    for idx, (start, end) in enumerate(epoch_ranges):
        # Get the relevant epoch range
        train_accuracy_range = accuracies['train'][start-1:end]
        val_accuracy_range = accuracies['val'][start-1:end]
        epoch_range = epochs[start-1:end]
        
        # Add bubbles for train accuracy
        for i, acc in enumerate(train_accuracy_range):
            x_data.append(epoch_range[i])
            y_data.append(acc)
            z_data.append(idx + 1)  # Different z values based on epoch range (1, 2, 3, 4)
            sizes.append(acc * 800)  # Bubble size based on accuracy
            colors.append(acc)  # Color based on accuracy
        
        # Add lines between bubbles (for train and validation accuracy)
        ax.plot(epoch_range, train_accuracy_range, zs=idx + 1, label=f'Train Accuracy (Epochs {start}-{end})', marker='o', linestyle='-', color='blue')
        ax.plot(epoch_range, val_accuracy_range, zs=idx + 1, label=f'Validation Accuracy (Epochs {start}-{end})', marker='x', linestyle='-', color='green')

    # Scatter plot with bubbles
    scatter = ax.scatter(x_data, y_data, z_data, s=sizes, c=colors, cmap='viridis', alpha=0.6)

    # Add color bar for accuracy
    plt.colorbar(scatter, ax=ax, label='Accuracy')

    # Add title and legend
    ax.set_title('Training and Validation Accuracy with 3D Bubbles', fontsize=16, fontweight='bold')
    ax.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_accuracy_with_3d_bubbles(history_model_net)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Importing 3D plotting module

def plot_learning_3d(history_model_net):
    """
    Plots training & validation accuracy and loss in 3D.
    """
    epochs = np.arange(1, len(history_model_net.history['accuracy']) + 1)
    
    # Create 3D plot
    fig = plt.figure(figsize=(14, 10))

    # First 3D subplot for accuracy
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot(epochs, history_model_net.history['accuracy'], zs=1, label='Train Accuracy', color='blue', linewidth=2)
    ax1.plot(epochs, history_model_net.history['val_accuracy'], zs=2, label='Validation Accuracy', color='green', linewidth=2)
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.set_zlabel("Type")
    ax1.set_title("Training and Validation Accuracy in 3D")
    ax1.legend()

    # Second 3D subplot for loss
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot(epochs, history_model_net.history['loss'], zs=1, label='Train Loss', color='red', linewidth=2)
    ax2.plot(epochs, history_model_net.history['val_loss'], zs=2, label='Validation Loss', color='purple', linewidth=2)
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.set_zlabel("Type")
    ax2.set_title("Training and Validation Loss in 3D")
    ax2.legend()

    # Show the plot
    plt.show()

# Call the function with your model history
plot_learning_3d(history_model_net)
